### Notebook for analyzing DFO historical flood impacts

In [3]:
import pandas as pd
import os

In [5]:
directory_path = os.path.join("..", "data", "results", "social_flood", "events")
directory_names = [name for name in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, name))]

In [25]:
# Combine into one dataframe
results = []
invalid = []
valid = []
for i in directory_names:
    event_id = i.split("_")[1] # pull DFO ID
    df = pd.read_csv(os.path.join(directory_path, i, f"DFO_{event_id}_results.csv"))
    if df.empty:
        invalid.append(event_id)
        continue
    valid.append(event_id)
    for index, row in df.iterrows():
        new_row = {
            "DFO_ID": event_id,
            "ISO3": row['ISO3'],
            "FE": row['FE'],
            "BQFE": row['BQFE'],
            "CI": row['CI'],
            "QR": row['QR']
        }
        results.append(new_row)

results_df = pd.DataFrame(results, columns=["DFO_ID", "ISO3", "FE", "BQFE", "CI", "QR"])

In [27]:
results_df

,DFO_ID,ISO3,FE,BQFE,CI,QR
0,3109,UZB,1.803577e+02,4.884056,-0.311175,0.000000
1,3109,TJK,1.680495e+01,0.000000,0.206299,NaN
2,3109,TKM,2.120217e+02,0.037877,0.259891,0.000000
3,3221,MAR,5.490733e+03,586.670877,0.045811,0.284976
4,3221,TUN,5.645317e+03,1022.067133,-0.025370,0.885591
...,...,...,...,...,...,...
1220,4156,KHM,2.908859e+00,2.848489,-0.623101,0.000000
1221,2592,PHL,2.101542e+06,265281.013711,-0.054411,0.864058
1222,2194,MOZ,4.936570e+02,18.521308,0.202660,1.831511
1223,2194,ZWE,1.248028e+01,0.203025,0.084748,0.000000


In [28]:
results_df.to_csv("DFO_results.csv")

In [30]:
threshold = 100000
filtered_df = results_df[results_df['FE'] >= threshold]
filtered_df.to_csv(f"DFO_results_exposure-gt-{threshold}.csv")